# Query Rightmove with LangChain | OpenAI | Python | bs4

In [8]:
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
import statistics
import requests
from bs4 import BeautifulSoup

In [16]:
url='https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E116&maxBedrooms=3&minBedrooms=3&maxPrice=325000&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords='
url_me='https://www.rightmove.co.uk/property-for-sale/find.html?minBedrooms=2&keywords=&sortType=2&minPrice=250000&viewType=LIST&channel=BUY&index=0&radius=0.0&locationIdentifier=USERDEFINEDAREA%5E%7B%22id%22%3A8289123%7D'


url_base = 'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=USERDEFINEDAREA%5E%7B%22id%22%3A8289123%7D&minBedrooms=2&minPrice=250000&index=24&propertyTypes=&mustHave=&dontShow=&furnishTypes=&keywords='

In [21]:
url_test = f"{url_base}&index={24 + 24}"
url_test

'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=USERDEFINEDAREA%5E%7B%22id%22%3A8289123%7D&minBedrooms=2&minPrice=250000&index=24&propertyTypes=&mustHave=&dontShow=&furnishTypes=&keywords=&index=48'

## The Tools

In [32]:
# USE THIS ONE
def details(start_index=24, increment=24, max_pages=40) -> float:
    headers = { 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0' }

    location_l = []
    address_l = []
    price_l = []

    for page in range(max_pages):
        # Modify the index for each page
        url = f'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=USERDEFINEDAREA%5E%7B%22id%22%3A8289123%7D&minBedrooms=2&minPrice=250000&index={start_index + page * increment}&propertyTypes=&mustHave=&dontShow=&furnishTypes=&keywords='
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to retrieve the page with index {start_index + page * increment}. Status code: {response.status_code}")
            continue

        # Scrape the content

        soup = BeautifulSoup(response.text, "html.parser")

        props = soup.find_all("div", class_="l-searchResult is-list")


        # If no properties are found, we assume we've reached the last page
        if not props:
            print("No more properties found. Ending search.")
            break

        for i in range(len(props)):
            prop = props[i]
            location = (
                prop.find("address", itemprop='address')
                .get_text()
                .replace(",","")
                .replace("\n","")
                .strip("£")
                .strip()
            )
            location_l.append(location)

            address = (
                prop.find("h2", class_="propertyCard-title")
                .get_text()
                .replace(",","")
                .replace("\n","")
                .strip("£")
                .strip()
            )
            address_l.append(address)

            price = (
                prop.find("div", class_="propertyCard-priceValue")
                .get_text()
                .replace(",","")
                .strip("£")
                .strip()
            )
            price_l.append(price)
    return location_l, address_l, price_l

In [33]:
location_l, address_l, price_l = details()

In [34]:
data = {
    'Location': location_l,
    'Address': address_l,
    'Price': price_l
}
df = pd.DataFrame(data)
# Clean and convert price to a numeric type for BigQuery
df['Price'] = df['Price'].str.extract('(\d+,\d+|\d+)').replace(",", "", regex=True).astype(float)

# Save the DataFrame to a CSV file
csv_file = 'properties.csv'
df.to_csv(csv_file, index=False)

print(f"Data saved to {csv_file}")

Data saved to properties.csv
